In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\khadi\\Desktop\\spark-3.3.0-bin-hadoop3'

here we have not  an explicit rating about each product , but we have an implicit one , we can extract either a user likes an X product by 
counting the number of time that user purchased that X product 


P(u,i)between (0,1)
R(u,i) in R


P(u,i)====>0 if R(u,i)=0
      ====>1 if R(u,i)>=1

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)


import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
# import implicit

import warnings
warnings.filterwarnings("ignore")

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
R           = pd.read_csv("./ALS_datasets/fill_forward_als.csv")



In [6]:
R.describe()

,CustomerCode,Quantity,itemCode
count,1.150000e+05,115000.000000,115000.000000
mean,1.052455e+06,1.330826,12.000000
std,5.452519e+03,1.349391,6.633278
min,1.044068e+06,1.000000,1.000000
25%,1.049965e+06,1.000000,6.000000
50%,1.052314e+06,1.000000,12.000000
75%,1.055044e+06,1.000000,18.000000
max,1.375586e+06,7.000000,23.000000


In [7]:
R.shape

(115000, 4)

In [14]:
maxV=R['Quantity'].max()

In [15]:
minV=R['Quantity'].min()

In [4]:
from pyspark.mllib.recommendation import ALS

In [5]:
from pyspark import SparkContext
from pyspark.sql import SQLContext 
sc = SparkContext("local", "App Name")
sql = SQLContext(sc)


In [8]:
spark_df = sql.createDataFrame(R)
(training, test) = spark_df.randomSplit([0.8, 0.2])

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [10]:
spark_df.select('Quantity').describe().toPandas()

,summary,Quantity
0,count,115000
1,mean,1.3308260869565218
2,stddev,1.3493913036065408
3,min,1
4,max,7


In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [16]:

from pyspark.ml.recommendation import ALS


def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
 
  
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            als = ALS(userCol="CustomerCode", itemCol="itemCode", coldStartStrategy="drop",implicitPrefs=True, seed=1, ratingCol="Quantity").setMaxIter(maxIter).setRank(rank).setRegParam(reg)
            # train ALS model
            model = als.fit(training)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(test)
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="Quantity",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)/(maxV-minV)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model
    

In [17]:
ranks=[1,2,4,10]
regul=[0.06,0.05,0.1,0.2]
tune_ALS(training,test,10,regul,ranks)

1 latent factors and regularization = 0.06: validation RMSE is 0.23380914837146716
1 latent factors and regularization = 0.05: validation RMSE is 0.23385186774136443
1 latent factors and regularization = 0.1: validation RMSE is 0.23426472167904325
1 latent factors and regularization = 0.2: validation RMSE is 0.23662711947520695
2 latent factors and regularization = 0.06: validation RMSE is 0.235280872242442
2 latent factors and regularization = 0.05: validation RMSE is 0.23581013473355505
2 latent factors and regularization = 0.1: validation RMSE is 0.2356674366747693
2 latent factors and regularization = 0.2: validation RMSE is 0.2365850852431497
4 latent factors and regularization = 0.06: validation RMSE is 0.25264229347263395
4 latent factors and regularization = 0.05: validation RMSE is 0.2548173781310852
4 latent factors and regularization = 0.1: validation RMSE is 0.24467130969331705
4 latent factors and regularization = 0.2: validation RMSE is 0.23667563148913232
10 latent facto

ALSModel: uid=ALS_59831648f548, rank=1

In [20]:
# als = ALS(maxIter=20, regParam=0.05, userCol="CustomerCode",  seed=1,itemCol="itemCode", ratingCol="Purchases",rank=5,
#           coldStartStrategy="drop",implicitPrefs=True,)
als = ALS(userCol="CustomerCode", itemCol="itemCode", seed=1, ratingCol="Quantity", 
          coldStartStrategy="drop",implicitPrefs=True).setMaxIter(10).setRank(1).setRegParam(0.06)
            
# model = als.fit(R)


In [21]:
model = als.fit(training)

In [24]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="Quantity",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse/6))


Root-mean-square error = 0.23380914837146716


In [26]:
# p=predictions.toPandas
pre=predictions.select('Quantity','prediction').toPandas()
pre.head(100)


,Quantity,prediction
0,7,0.789014
1,1,0.700563
2,1,0.698833
3,1,0.702992
4,1,0.709655
...,...,...
95,1,0.772311
96,1,0.842338
97,1,0.836026
98,1,0.844574


In [27]:
pre.describe()

,Quantity,prediction
count,22957.000000,22957.000000
mean,1.326044,0.858366
std,1.338744,0.050916
min,1.000000,0.663164
25%,1.000000,0.828058
50%,1.000000,0.859787
75%,1.000000,0.889054
max,7.000000,1.078543


In [28]:
Recom=predictions.rdd.map(lambda p: Row(user=p[0],ProductPredictions=(p[4]),name=p[1])).toDF()


In [29]:
Recom=Recom.sort('user')
Recom.show(10)

+-------+------------------+--------------------+
|   user|ProductPredictions|                name|
+-------+------------------+--------------------+
|1044068| 0.908418595790863|          P_eAccount|
|1044068|0.8771370649337769|        P_securities|
|1044068|0.8796070218086243|       P_HomeAccount|
|1044068|0.8809033036231995|          P_Mortgage|
|1044069|0.8599770069122314|             P_Loans|
|1044069|0.8551916480064392|     P_SavingAccount|
|1044069|0.8535797595977783|             P_Funds|
|1044069|0.8609153628349304|P_MásParticularAc...|
|1044069|0.8591799139976501|          P_Pensions|
|1044072|0.9075520634651184|        P_CreditCard|
+-------+------------------+--------------------+
only showing top 10 rows



In [30]:
Recom.toPandas()


,user,ProductPredictions,name
0,1044068,0.879607,P_HomeAccount
1,1044068,0.880903,P_Mortgage
2,1044068,0.908419,P_eAccount
3,1044068,0.877137,P_securities
4,1044069,0.853580,P_Funds
...,...,...,...
22952,1057215,0.886158,P_eAccount
22953,1375586,0.878432,P_Funds
22954,1375586,0.883660,P_Garantees
22955,1375586,0.882441,P_ParticularPlusAccount


In [31]:
Rec=Recom.toPandas()

Rec['ProductPredictions'].describe()

count    22957.000000
mean         0.858361
std          0.050917
min          0.663164
25%          0.828058
50%          0.859787
75%          0.889054
max          1.078543
Name: ProductPredictions, dtype: float64

In [32]:
Rec[Rec.user==1050088]

,user,ProductPredictions,name
6232,1050088,0.833388,P_Derivada_Account
6233,1050088,0.835894,P_HomeAccount
6234,1050088,0.835837,P_MeduimTermDeposits
6235,1050088,0.837126,P_Mortgage
6236,1050088,0.835125,P_ParticularPlusAccount
6237,1050088,0.832900,P_SavingAccount


In [33]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each product
ProductRecs = model.recommendForAllItems(10)

In [34]:
# Generate top 10 product recommendations for a specified set of users
users = spark_df.select(als.getUserCol()).where('CustomerCode=1375586')
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of product
items = spark_df.select(als.getItemCol()).distinct().limit(3)
itemSubSetRecs = model.recommendForItemSubset(items, 10)

In [35]:
userSubsetRecs.head(10)
# typeof(userSubsetRecs)

[Row(CustomerCode=1375586, recommendations=[Row(itemCode=3, rating=0.9917897582054138), Row(itemCode=23, rating=0.9135057926177979), Row(itemCode=13, rating=0.9121853113174438), Row(itemCode=5, rating=0.9005982279777527), Row(itemCode=12, rating=0.8926946520805359), Row(itemCode=6, rating=0.8920359015464783), Row(itemCode=22, rating=0.8910372257232666), Row(itemCode=19, rating=0.8903310894966125), Row(itemCode=18, rating=0.8896164298057556), Row(itemCode=8, rating=0.8881850838661194)])]